In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./csv_folder/table2.csv")

In [3]:
df.head()

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,intime,chartoffset
0,200003,2199-08-02 19:15:00,132.0,NaN,NaN,NaN,25.0,39.555554,97.0,NaN,2199-08-02 19:50:04,-35.066667
1,200003,2199-08-02 20:00:00,119.0,NaN,NaN,NaN,35.0,NaN,97.0,NaN,2199-08-02 19:50:04,9.933333
2,200003,2199-08-02 21:00:00,122.0,NaN,NaN,NaN,34.0,38.999998,97.0,NaN,2199-08-02 19:50:04,69.933333
3,200003,2199-08-02 21:25:00,115.0,NaN,NaN,NaN,33.0,NaN,96.0,NaN,2199-08-02 19:50:04,94.933333
4,200003,2199-08-02 21:30:00,116.0,NaN,NaN,NaN,34.0,NaN,95.0,NaN,2199-08-02 19:50:04,99.933333


In [4]:
df_group = df.groupby("icustay_id")

In [5]:
def calc_percent(df, timerange=[0, 1440], col='sysbp', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if num_nonna==0:
        return np.nan
    value_min, value_max = valuerange[0], valuerange[1]
    range_num = non_na[(non_na[col]>=value_min)].shape[0]
    return range_num/num_nonna

In [69]:
def calc_percent_below(df, timerange=[0, 1440], col='sysbp', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if ((num_nonna==0) ):
        return np.nan
    value_min= valuerange[0]
    below_num = non_na[(non_na[col]<value_min)].shape[0]
    return below_num/num_nonna

In [70]:
def calc_percent_above(df, timerange=[0, 1440], col='sysbp', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if ((num_nonna==0)  ):
        return np.nan
    value_max= valuerange[1]
    above_num = non_na[(non_na[col]>value_max)].shape[0]
    return above_num/num_nonna

In [6]:
in_range_48h = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='meanbp', valuerange=[75, 75]))

In [8]:
in_range_48h.head(1)

icustay_id
200003    0.607843
dtype: float64

In [9]:
in_range_48h = pd.DataFrame(in_range_48h).reset_index()
in_range_48h.columns = ["icustay_id","prop_map_48_AKI"]
result_df = in_range_48h.copy()
result_df["propBelow_map_48_AKI"] = 1-result_df['prop_map_48_AKI']

In [10]:
in_range_72h = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='meanbp', valuerange=[76,76]))
in_range_72h = pd.DataFrame(in_range_72h).reset_index()
in_range_72h.columns = ["icustay_id", "prop_map_72_AKI"]

result_df['prop_map_72_AKI'] = in_range_72h['prop_map_72_AKI']
result_df['propBelow_map_72_AKI'] = 1-result_df['prop_map_72_AKI']

In [13]:
result_df.head()

,icustay_id,prop_map_48_AKI,propBelow_map_48_AKI,prop_map_72_AKI,propBelow_map_72_AKI
0,200003,0.607843,0.392157,0.675325,0.324675
1,200007,NaN,NaN,NaN,NaN
2,200009,0.581818,0.418182,0.527273,0.472727
3,200012,NaN,NaN,NaN,NaN
4,200014,0.790698,0.209302,0.767442,0.232558


In [14]:
result_df.to_csv("./csv_folder/mimic_prop_map_48h_72h.csv")

In [84]:
sbp_24_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='sysbp', valuerange=[112,125]))
sbp_24_below = pd.DataFrame(sbp_24_below).reset_index()
sbp_24_below.columns = ["icustay_id", "propBelow_asp_24"]
sbp_24_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='sysbp', valuerange=[112,125]))
sbp_24_above = pd.DataFrame(sbp_24_above).reset_index()
sbp_24_above.columns = ["icustay_id", "propAbove_asp_24"]
result_df['propBelow_asp_24'] = sbp_24_below['propBelow_asp_24']
result_df['propAbove_asp_24'] = sbp_24_above['propAbove_asp_24']
result_df['prop_asp_24'] = 1-result_df['propAbove_asp_24'] -result_df['propBelow_asp_24']

In [77]:
result_df.head()

,icustay_id,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24
0,200003,0.058824,0.882353,0.058824,0.687500,0.312500,0.000000e+00
1,200007,NaN,NaN,NaN,NaN,NaN,NaN
2,200009,0.645161,0.354839,0.000000,0.774194,0.225806,0.000000e+00
3,200012,NaN,NaN,NaN,NaN,NaN,NaN
4,200014,0.250000,0.678571,0.071429,0.321429,0.678571,-5.551115e-17


In [85]:
adp_24_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='diasbp', valuerange=[50,61]))
adp_24_below = pd.DataFrame(adp_24_below).reset_index()
adp_24_below.columns = ["icustay_id", "propBelow_adp_24"]
adp_24_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='diasbp', valuerange=[50,61]))
adp_24_above = pd.DataFrame(adp_24_above).reset_index()
adp_24_above.columns = ["icustay_id", "propAbove_adp_24"]
result_df['propBelow_adp_24'] = adp_24_below['propBelow_adp_24']
result_df['propAbove_adp_24'] = adp_24_above['propAbove_adp_24']
result_df['prop_adp_24'] = 1-result_df['propAbove_adp_24'] -result_df['propBelow_adp_24']

In [79]:
result_df.head()

,icustay_id,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
0,200003,0.058824,0.882353,0.058824,0.687500,0.312500,0.000000e+00,0.0,0.937500,0.062500
1,200007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200009,0.645161,0.354839,0.000000,0.774194,0.225806,0.000000e+00,0.0,0.870968,0.129032
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200014,0.250000,0.678571,0.071429,0.321429,0.678571,-5.551115e-17,0.0,0.821429,0.178571


In [86]:
result_df.to_csv("./csv_folder/mimic_prop_24h_test.csv")

In [68]:
mbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_mean', valuerange=[78.27889, 98.53015]))
mbp_48_inrange = pd.DataFrame(mbp_48_inrange).reset_index()
mbp_48_inrange.columns = ["icustay_id", "mbp_48_inrange"]
result_df['mbp_48_inrange'] = mbp_48_inrange['mbp_48_inrange']

sbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_systolic', valuerange=[122.4724, 154.3116]))
sbp_48_inrange = pd.DataFrame(sbp_48_inrange).reset_index()
sbp_48_inrange.columns = ["icustay_id", "sbp_48_inrange"]
result_df['sbp_48_inrange'] = sbp_48_inrange['sbp_48_inrange']

dbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_diastolic', valuerange=[60.39196, 79.55276]))
dbp_48_inrange = pd.DataFrame(dbp_48_inrange).reset_index()
dbp_48_inrange.columns = ["icustay_id", "dbp_48_inrange"]
result_df['dbp_48_inrange'] = dbp_48_inrange['dbp_48_inrange']

mbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_mean', valuerange=[79.39698, 93.25628]))
mbp_72_inrange = pd.DataFrame(mbp_72_inrange).reset_index()
mbp_72_inrange.columns = ["icustay_id", "mbp_72_inrange"]
result_df['mbp_72_inrange'] = mbp_72_inrange['mbp_72_inrange']

sbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_systolic', valuerange=[123.4673, 153.3166]))
sbp_72_inrange = pd.DataFrame(sbp_72_inrange).reset_index()
sbp_72_inrange.columns = ["icustay_id", "sbp_72_inrange"]
result_df['sbp_72_inrange'] = sbp_72_inrange['sbp_72_inrange']

dbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_diastolic', valuerange=[61.01005, 82.64322]))
dbp_72_inrange = pd.DataFrame(dbp_72_inrange).reset_index()
dbp_72_inrange.columns = ["icustay_id", "dbp_72_inrange"]
result_df['dbp_72_inrange'] = dbp_72_inrange['dbp_72_inrange']

In [69]:
result_df.head()

,patientunitstayid,mbp_24_inrange,sbp_24_inrange,dbp_24_inrange,mbp_48_inrange,sbp_48_inrange,dbp_48_inrange,mbp_72_inrange,sbp_72_inrange,dbp_72_inrange
0,141168,0.700000,NaN,NaN,0.533333,0.200000,0.200000,0.466667,0.200000,0.200000
1,141194,0.000000,0.041667,0.000000,0.153846,0.098039,0.078431,0.153846,0.098039,0.078431
2,141203,0.417582,NaN,NaN,0.316832,NaN,NaN,0.247525,NaN,NaN
3,141229,1.000000,NaN,NaN,0.500000,NaN,NaN,0.500000,NaN,NaN
4,141233,0.364486,0.457944,0.037383,0.131068,0.150485,0.009709,0.122881,0.114407,0.084746


In [73]:
result_df[result_df['mbp_24_inrange'].notnull()].shape

(20884, 10)

In [12]:
df1 = pd.read_csv("./csv_folder/mimic_clean_19Aug.csv", index_col=0)

In [13]:
df1.head()

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,los_hospital,age,ethnicity,admission_type,...,exclude_tempc,ibp_systolic_24h,ibp_systolic_48h,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h
0,55853,100010,271147,F,2109-12-10 07:15:00,2109-12-14 16:45:00,4.3958,54.5208,WHITE,ELECTIVE,...,False,100.5,99.0,99.0,47.0,50.0,50.0,66.5,68.0,68.0
1,87977,100011,214619,M,2177-08-29 04:51:00,2177-09-12 14:24:00,14.3979,21.5041,HISPANIC OR LATINO,EMERGENCY,...,False,127.0,147.0,148.0,67.0,72.0,72.0,84.0,91.0,92.0
2,14121,100058,297937,F,2139-09-08 21:38:00,2139-09-26 13:55:00,17.6785,56.8114,WHITE,EMERGENCY,...,False,108.0,108.0,108.0,59.0,58.0,57.0,79.0,78.0,77.0
3,20683,100186,203786,M,2168-01-07 19:33:00,2168-01-12 14:00:00,4.7688,19.8138,WHITE,EMERGENCY,...,False,122.0,118.5,118.5,59.5,59.0,59.0,77.0,77.0,77.0
4,81865,100215,248622,M,2193-05-03 20:47:00,2193-05-15 18:31:00,11.9056,48.0116,WHITE,EMERGENCY,...,False,113.5,113.0,113.0,69.0,66.0,66.0,82.0,80.0,81.0


In [15]:
df3 = pd.merge(df1, result_df, how='left', on=['icustay_id'])

In [16]:
df3.head()

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,los_hospital,age,ethnicity,admission_type,...,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h,mbp_24_inrange,sbp_24_inrange,dbp_24_inrange
0,55853,100010,271147,F,2109-12-10 07:15:00,2109-12-14 16:45:00,4.3958,54.5208,WHITE,ELECTIVE,...,99.0,47.0,50.0,50.0,66.5,68.0,68.0,0.166667,0.208333,0.041667
1,87977,100011,214619,M,2177-08-29 04:51:00,2177-09-12 14:24:00,14.3979,21.5041,HISPANIC OR LATINO,EMERGENCY,...,148.0,67.0,72.0,72.0,84.0,91.0,92.0,0.413793,0.379310,0.068966
2,14121,100058,297937,F,2139-09-08 21:38:00,2139-09-26 13:55:00,17.6785,56.8114,WHITE,EMERGENCY,...,108.0,59.0,58.0,57.0,79.0,78.0,77.0,0.636364,0.212121,0.000000
3,20683,100186,203786,M,2168-01-07 19:33:00,2168-01-12 14:00:00,4.7688,19.8138,WHITE,EMERGENCY,...,118.5,59.5,59.0,59.0,77.0,77.0,77.0,0.750000,0.583333,0.000000
4,81865,100215,248622,M,2193-05-03 20:47:00,2193-05-15 18:31:00,11.9056,48.0116,WHITE,EMERGENCY,...,113.0,69.0,66.0,66.0,82.0,80.0,81.0,0.818182,0.363636,0.090909


In [90]:
# df3.plot(x='mbp_24_inrange', y='hosp_mort')
# df3['hospital_mortality']= 1
# df3.loc[df3['hosp_mort']=='ALIVE', 'hospital_mortality']=0

In [91]:
# df3[['hosp_mort','hospital_mortality']].head(2)

,hosp_mort,hospital_mortality
0,EXPIRED,1
1,ALIVE,0


In [17]:
df3.to_csv("./csv_folder/mimic_table1_wth_range.csv")